In [32]:
import flax 
import flax.linen as nn
from flax.training import train_state, checkpoints
from flax.core.frozen_dict import unfreeze, freeze

from utils.dataloader import mel_dataset
from torch.utils.data import DataLoader, random_split

import jax
import numpy as np
import jax.numpy as jnp
import optax

from torch.utils.tensorboard import SummaryWriter
import os
from utils.config_hook import yaml_config_hook
from tqdm import tqdm

import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile


2022-09-22 01:27:03.015774: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2022-09-22 01:27:03.045919: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-22 01:27:03.716109: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2022-09-22 01:27:03.716262: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib


In [2]:
cd

/home/anthonypark6904


In [33]:
data = mel_dataset('/mnt/disks/sdb/dataset', 'total')

Load song_meta.json...


100%|███████████████████████████████| 707989/707989 [00:01<00:00, 469713.55it/s]


Load complete!

Load file list...


710it [01:20,  8.83it/s]


In [14]:
cd /mnt/disks/sdb/dataset

/mnt/disks/sdb/dataset


In [15]:
ls

arena_mel/  song_meta.json


In [34]:
data.genre_index

{'GN0900': 0,
 'GN1600': 1,
 'GN1100': 2,
 'GN1800': 3,
 'GN1700': 4,
 'GN0300': 5,
 'GN0100': 6,
 'GN1200': 7,
 'GN1300': 8,
 'GN2700': 9,
 'GN2500': 10,
 'GN0200': 11,
 'GN2800': 12,
 'GN0500': 13,
 'GN0800': 14,
 'GN1900': 15,
 'GN1500': 16,
 'GN2600': 17,
 'GN1000': 18,
 'GN1400': 19,
 'GN2000': 20,
 'GN2100': 21,
 'GN2200': 22,
 'GN0600': 23,
 'GN0700': 24,
 'GN0400': 25,
 'GN2900': 26,
 'GN9000': 27,
 'GN2300': 28,
 'GN2400': 29}

In [35]:
data.count_dict

{'GN0900': 64724,
 'GN1600': 56203,
 'GN1000': 52786,
 'GN1100': 49699,
 'GN1700': 48143,
 'GN1800': 42335,
 'GN0100': 41121,
 'GN1200': 34851,
 'GN1300': 31825,
 'GN1500': 27244,
 'GN1900': 21096,
 'GN2100': 19958,
 'GN0300': 16437,
 'GN2200': 14463,
 'GN2000': 13189,
 'GN1400': 12792,
 'GN0700': 12457,
 'GN0600': 11893,
 'GN0200': 11591,
 'GN0400': 10551,
 'GN0800': 5921,
 'GN2600': 3709,
 'GN2400': 2222,
 'GN9000': 1824,
 'GN2800': 1780,
 'GN2700': 1335,
 'GN2300': 1130,
 'GN0500': 16}

In [6]:
genre_key = {
  "GN0100": "발라드",
  "GN0101": "세부장르전체",
  "GN0102": "'80",
  "GN0103": "'90",
  "GN0104": "'00",
  "GN0105": "'10-",
  "GN0200": "댄스",
  "GN0201": "세부장르전체",
  "GN0202": "'80",
  "GN0203": "'90",
  "GN0204": "'00",
  "GN0205": "'10-",
  "GN0300": "랩/힙합",
  "GN0301": "세부장르전체",
  "GN0302": "랩 스타일",
  "GN0303": "보컬 스타일",
  "GN0304": "언더그라운드 힙합",
  "GN0305": "시대별",
  "GN0400": "R&B/Soul",
  "GN0401": "세부장르전체",
  "GN0402": "어반",
  "GN0403": "R&B",
  "GN0500": "인디음악",
  "GN0501": "세부장르전체",
  "GN0502": "포크",
  "GN0503": "록",
  "GN0504": "일렉",
  "GN0505": "힙합",
  "GN0506": "발라드",
  "GN0507": "'90",
  "GN0508": "'00",
  "GN0509": "'10-",
  "GN0600": "록/메탈",
  "GN0601": "세부장르전체",
  "GN0602": "'70",
  "GN0603": "'80",
  "GN0604": "'90",
  "GN0605": "'00",
  "GN0606": "'10-",
  "GN0700": "성인가요",
  "GN0701": "세부장르전체",
  "GN0702": "신세대트로트",
  "GN0703": "전설의트로트",
  "GN0704": "뽕짝트로트",
  "GN0705": "트로트메들리",
  "GN0706": "트로트'60-'70",
  "GN0707": "트로트'80-'90",
  "GN0708": "트로트'00-",
  "GN0709": "성인가요'80-'90",
  "GN0710": "성인가요'00-",
  "GN0800": "포크/블루스",
  "GN0801": "세부장르전체",
  "GN0802": "'60-'70",
  "GN0803": "'80-'90",
  "GN0804": "'00",
  "GN0805": "'10-",
  "GN0900": "POP",
  "GN0901": "세부장르전체",
  "GN0902": "얼터너티브팝",
  "GN0903": "올디스",
  "GN0904": "월드팝",
  "GN0905": "'60-'70",
  "GN0906": "'80-'90",
  "GN0907": "'00",
  "GN0908": "'10-",
  "GN1000": "록/메탈",
  "GN1001": "세부장르전체",
  "GN1002": "모던록",
  "GN1003": "얼터너티브록",
  "GN1004": "프로그레시브/아트록",
  "GN1005": "하드록",
  "GN1006": "헤비메탈",
  "GN1007": "뉴메탈/하드코어",
  "GN1008": "포스트록",
  "GN1009": "'60",
  "GN1010": "'70",
  "GN1011": "'80",
  "GN1012": "'90",
  "GN1013": "'00",
  "GN1014": "'10-",
  "GN1100": "일렉트로니카",
  "GN1101": "세부장르전체",
  "GN1102": "일렉트로니카",
  "GN1103": "하우스",
  "GN1104": "클럽뮤직",
  "GN1105": "일렉트로닉팝",
  "GN1106": "'80",
  "GN1107": "'90",
  "GN1108": "'00",
  "GN1109": "'10-",
  "GN1200": "랩/힙합",
  "GN1201": "세부장르전체",
  "GN1202": "팝랩",
  "GN1203": "얼터너티브힙합",
  "GN1204": "갱스터/하드코어랩",
  "GN1205": "East&West",
  "GN1206": "'80",
  "GN1207": "'90",
  "GN1208": "'00",
  "GN1209": "'10-",
  "GN1300": "R&B/Soul",
  "GN1301": "세부장르전체",
  "GN1302": "컨템포러리 R&B",
  "GN1303": "소울",
  "GN1304": "어반",
  "GN1305": "'60-'70",
  "GN1306": "'80-'90",
  "GN1307": "'00",
  "GN1308": "'10-",
  "GN1400": "포크/블루스/컨트리",
  "GN1401": "세부장르전체",
  "GN1402": "포크",
  "GN1403": "블루스",
  "GN1404": "컨트리",
  "GN1405": "'60-'70",
  "GN1406": "'80-'90",
  "GN1407": "'00-",
  "GN1500": "OST",
  "GN1501": "세부장르전체",
  "GN1502": "국내영화",
  "GN1503": "국외영화",
  "GN1504": "국내드라마",
  "GN1505": "국외드라마",
  "GN1506": "애니메이션/게임",
  "GN1507": "국내뮤지컬",
  "GN1508": "국외뮤지컬",
  "GN1509": "애니메이션/웹툰",
  "GN1510": "게임",
  "GN1600": "클래식",
  "GN1601": "세부장르전체",
  "GN1602": "관현악",
  "GN1603": "교향곡",
  "GN1604": "실내악",
  "GN1605": "협주곡",
  "GN1606": "독주곡",
  "GN1607": "오페라",
  "GN1608": "크로스오버",
  "GN1609": "현대음악",
  "GN1610": "성악/합창곡",
  "GN1611": "발레/무용",
  "GN1612": "지휘/연주자",
  "GN1613": "컴필레이션",
  "GN1614": "교향/관현악",
  "GN1615": "오페라/성악",
  "GN1700": "재즈",
  "GN1701": "세부장르전체",
  "GN1702": "보컬재즈",
  "GN1703": "애시드/퓨전/팝",
  "GN1704": "Bop",
  "GN1705": "보사노바",
  "GN1706": "J-Jazz",
  "GN1707": "라틴재즈",
  "GN1708": "빅밴드/스윙",
  "GN1709": "악기별",
  "GN1710": "컴필레이션",
  "GN1800": "뉴에이지",
  "GN1801": "세부장르전체",
  "GN1802": "이지리스닝",
  "GN1803": "J-Newage",
  "GN1804": "기능성음악",
  "GN1805": "기타",
  "GN1806": "피아노",
  "GN1807": "컴필레이션",
  "GN1900": "J-POP",
  "GN1901": "세부장르전체",
  "GN1902": "POP",
  "GN1903": "록",
  "GN1904": "일렉트로니카",
  "GN1905": "랩/힙합",
  "GN1906": "R&B/Soul",
  "GN1907": "시부야케이",
  "GN1908": "뉴에이지",
  "GN1909": "재즈",
  "GN1910": "힙합/R&B",
  "GN1911": "'80-'90",
  "GN1912": "'00-'10",
  "GN2000": "월드뮤직",
  "GN2001": "세부장르전체",
  "GN2002": "샹송/프렌치팝",
  "GN2003": "칸초네/이탈리안팝",
  "GN2004": "중국음악",
  "GN2005": "켈틱/아이리시",
  "GN2006": "브라질",
  "GN2007": "탱고/플라멩코",
  "GN2008": "라틴",
  "GN2009": "레게",
  "GN2010": "파두",
  "GN2100": "CCM",
  "GN2101": "세부장르전체",
  "GN2102": "국내CCM",
  "GN2103": "국외CCM",
  "GN2104": "워십",
  "GN2105": "찬송가",
  "GN2106": "성가",
  "GN2107": "연주곡",
  "GN2108": "어린이",
  "GN2109": "성경",
  "GN2110": "NEW",
  "GN2200": "어린이/태교",
  "GN2201": "유아동요",
  "GN2202": "창작동요",
  "GN2203": "영어동요",
  "GN2204": "영어동요",
  "GN2205": "영어동요",
  "GN2206": "영어동화",
  "GN2207": "만화",
  "GN2208": "자장가",
  "GN2209": "태교동화",
  "GN2210": "태교",
  "GN2211": "릴렉싱&힐링",
  "GN2212": "명작동화",
  "GN2213": "교과서동요",
  "GN2214": "전래동요",
  "GN2215": "창작동화",
  "GN2216": "어린이클래식",
  "GN2300": "종교음악",
  "GN2301": "가톨릭",
  "GN2302": "불교",
  "GN2400": "국악",
  "GN2401": "세부장르전체",
  "GN2402": "크로스오버",
  "GN2403": "국악가요",
  "GN2404": "민요",
  "GN2405": "판소리/단가",
  "GN2406": "풍물/사물놀이",
  "GN2500": "아이돌",
  "GN2501": "세부장르전체",
  "GN2502": "남자 아이돌",
  "GN2503": "여자 아이돌",
  "GN2504": "랩/힙합",
  "GN2505": "발라드",
  "GN2506": "댄스",
  "GN2600": "일렉트로니카",
  "GN2601": "세부장르전체",
  "GN2602": "댄스 스타일",
  "GN2603": "팝 스타일",
  "GN2604": "시대별",
  "GN2700": "EDM",
  "GN2701": "세부장르전체",
  "GN2702": "하우스",
  "GN2703": "클럽",
  "GN2704": "DJ",
  "GN2800": "뮤직테라피",
  "GN2801": "세부장르전체",
  "GN2802": "ASMR/자연",
  "GN2803": "힐링/명상/요가",
  "GN2804": "집중력",
  "GN2805": "숙면유도",
  "GN2806": "반려동물",
  "GN2900": "뮤지컬",
  "GN2901": "세부장르전체",
  "GN2902": "국내뮤지컬",
  "GN2903": "국외뮤지컬",
  "GN3000": "크리스마스",
  "GN9000":"nan"
}


In [7]:
linear = np.load('linear_array.npy')

FileNotFoundError: [Errno 2] No such file or directory: 'linear_array.npy'

In [ ]:
data.genre_keys

In [ ]:
dic = {k:genre_key[v] for k,v in data.genre_keys.items()}
dic 

In [ ]:
data.sort_dict

In [ ]:
# count = 0
answer = 0
inference_genre = {}
answer_genre = {}
etc = 0
for i in linear:
#     print(f'inference : {np.argmax(i[0])}, {genre_key[data.genre_keys[np.argmax(i[0])]]}')
#     print(f'Answer : {np.argmax(i[1])}, {genre_key[data.genre_keys[np.argmax(i[1])]]}\n')
    
    if np.argmax(i[1])== np.argmax(i[0]):
        answer += 1
    try:    
        inference_genre[genre_key[data.genre_keys[np.argmax(i[0])]]] += 1
    except:
        try:
            inference_genre[genre_key[data.genre_keys[np.argmax(i[0])]]] = 0 
        except:
            etc += 1
            # print(data.genre_keys[np.argmax(i[1])])
            # print(i[1])
            # print(i[0])
            # print(np.argmax(i[0]))
        
    try:    
        answer_genre[genre_key[data.genre_keys[np.argmax(i[1])]]] += 1
    except:
        try:
            answer_genre[genre_key[data.genre_keys[np.argmax(i[1])]]] = 0 
        except:
            etc += 1
            # print(data.genre_keys[np.argmax(i[1])])
            # print(i[1])
            # print(i[0])
            # print(np.argmax(i[0]))
print(f'etc count {etc}')
    # if count == 100:
        # break

In [ ]:
print(inference_genre)
print()
print(answer_genre)

In [ ]:
total_diff = {}
for key in inference_genre:
    total_diff[key] = inference_genre[key] - answer_genre[key]

In [ ]:
total_diff

In [ ]:

"GN1500": "OST",
"GN2100": "CCM”,  
"GN1900": "J-POP",
"GN9000":"nan"
"GN2000": "월드뮤직",
'GN2300'
'GN2800'


In [ ]:
data.sort_dict

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.bar(range(len(inference_genre)), inference_genre.values())

In [ ]:
inference_genre.keys()

In [ ]:
answer_genre

In [ ]:
del answer_genre['종교음악']

In [ ]:
answer_genre

In [ ]:
label = [data.genre_keys[np.argmax(y)] for x, y in data]                                       
writer = SummaryWriter('SimCLR_64_linear')
writer.add_embedding(embed_list, metadata=label)
writer.close()

In [ ]:
data.sort_dict

In [ ]:
f = data.sort_dict

In [ ]:
f

In [ ]:
drop = ('GN9000', 'GN2300')

In [ ]:
f

In [ ]:
drop = ('GN9000', 'GN2300')

for key in drop:
    f.pop(key, None)

In [ ]:
f

In [ ]:
drop = ('GN1500', 'GN2100','GN1900','GN9000','GN2000','GN2300','GN2800')

In [ ]:
drop = ('GN1500', 'GN2100','GN1900','GN9000','GN2000','GN2300','GN2800')
for key in drop:
    data.sort_dict.pop(key, None)

In [31]:
data.sort_dict

NameError: name 'data' is not defined

In [ ]:
"GN1500": "OST",
"GN2100": "CCM”,  
"GN1900": "J-POP",
"GN9000":"nan"
"GN2000": "월드뮤직",
'GN2300'
'GN2800'

In [30]:
data.sort_dict

{'GN0900': 0,
 'GN1600': 1,
 'GN1000': 2,
 'GN1100': 3,
 'GN1700': 4,
 'GN1800': 5,
 'GN0100': 6,
 'GN1200': 7,
 'GN1300': 8,
 'GN1500': 9,
 'GN1900': 10,
 'GN2100': 11,
 'GN0300': 12,
 'GN2200': 13,
 'GN2000': 14,
 'GN1400': 15,
 'GN0700': 16,
 'GN0600': 17,
 'GN0200': 18,
 'GN0400': 19,
 'GN0800': 20,
 'GN2600': 21,
 'GN2400': 22,
 'GN9000': 23,
 'GN2800': 24,
 'GN2700': 25,
 'GN2300': 26,
 'GN0500': 27}

In [12]:
genre_dict = [genre_key[k] for k,v in data.count_dict.items()]

In [13]:
genre_dict

['POP',
 '클래식',
 '록/메탈',
 '일렉트로니카',
 '재즈',
 '뉴에이지',
 '발라드',
 '랩/힙합',
 'R&B/Soul',
 'OST',
 'J-POP',
 'CCM',
 '랩/힙합',
 '어린이/태교',
 '월드뮤직',
 '포크/블루스/컨트리',
 '성인가요',
 '록/메탈',
 '댄스',
 'R&B/Soul',
 '포크/블루스',
 '일렉트로니카',
 '국악',
 'nan',
 '뮤직테라피',
 'EDM',
 '종교음악',
 '인디음악']

In [5]:
import os

In [6]:
meta_file_path = os.path.join('/mnt/disks/sdb/dataset', "song_meta.json")
if os.path.isfile(meta_file_path):
    with open(meta_file_path) as f:
        song_meta = json.load(f)

In [10]:
import json
import pandas as pd

In [11]:
song_meta = pd.DataFrame(song_meta)

In [12]:
song_meta

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4
...,...,...,...,...,...,...,...,...,...
707984,[GN2001],19991219,The Best Best Of The Black President,65254,[166499],Coffin For Head Of State,[GN2000],[Fela Kuti],707984
707985,[GN0901],19860000,True Colors,44141,[11837],Change Of Heart,[GN0900],[Cyndi Lauper],707985
707986,"[GN0105, GN0101]",20160120,행보 2015 윤종신 / 작사가 윤종신 Live Part.1,2662866,[437],스치듯 안녕,[GN0100],[윤종신],707986
707987,"[GN1807, GN1801]",20131217,명상의 시간을 위한 뉴에이지 음악,2221722,[729868],숲의 빛,[GN1800],[Nature Piano],707987


In [66]:
f['song_gn_gnr_basket']

0         [GN0900]
1         [GN1600]
2         [GN0900]
3         [GN1100]
4         [GN1800]
            ...   
707984    [GN2000]
707985    [GN0900]
707986    [GN0100]
707987    [GN1800]
707988    [GN0600]
Name: song_gn_gnr_basket, Length: 707989, dtype: object

In [ ]:
"GN1500": "OST",
"GN2100": "CCM”,  
"GN1900": "J-POP",
"GN9000":"nan"
"GN2000": "월드뮤직",
'GN2300' 
'GN2800'

In [16]:
#GN1500
ost_index = []
for i in range(len(song_meta['song_gn_gnr_basket'])):
    if song_meta['song_gn_gnr_basket'][i]==['GN0600']:
        ost_index.append(i)

In [18]:
len(ost_index)

11920

In [42]:
len(song_meta['song_gn_gnr_basket'])

707989

In [19]:
#GN2100
ccm_index = []
for i in range(len(song_meta['song_gn_gnr_basket'])):
    if song_meta['song_gn_gnr_basket'][i]==['GN2100']:
        ccm_index.append(i)

In [20]:
len(ccm_index)

19993

In [21]:
#GN9000
nan_index = []
for i in range(len(song_meta['song_gn_gnr_basket'])):
    if song_meta['song_gn_gnr_basket'][i]==['GN9000']:
        nan_index.append(i)

In [22]:
len(nan_index)

1834

In [24]:
#GN2000
world_music_index = []
for i in range(len(song_meta['song_gn_gnr_basket'])):
    if song_meta['song_gn_gnr_basket'][i]==['GN2000']:
        world_music_index.append(i)

In [25]:
len(world_music_index)

13192

In [29]:
#GN2800
Music_Therapy_index = []
for i in range(len(song_meta['song_gn_gnr_basket'])):
     if song_meta['song_gn_gnr_basket'][i]==['GN2800']:
        Music_Therapy_index.append(i)

In [30]:
len(Music_Therapy_index)

1783

In [40]:
#GN2300 종교음악

GN2300_index = []

for i in range(len(song_meta['song_gn_gnr_basket'])):
     if song_meta['song_gn_gnr_basket'][i]==['GN2300']:
        GN2300_index.append(i)

In [41]:
len(GN2300_index)

1135

In [39]:
data.count_dict

{'GN0900': 64724,
 'GN1600': 56203,
 'GN1000': 52786,
 'GN1100': 49699,
 'GN1700': 48143,
 'GN1800': 42335,
 'GN0100': 41121,
 'GN1200': 34851,
 'GN1300': 31825,
 'GN1500': 27244,
 'GN1900': 21096,
 'GN2100': 19958,
 'GN0300': 16437,
 'GN2200': 14463,
 'GN2000': 13189,
 'GN1400': 12792,
 'GN0700': 12457,
 'GN0600': 11893,
 'GN0200': 11591,
 'GN0400': 10551,
 'GN0800': 5921,
 'GN2600': 3709,
 'GN2400': 2222,
 'GN9000': 1824,
 'GN2800': 1780,
 'GN2700': 1335,
 'GN2300': 1130,
 'GN0500': 16}